In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
from torch import nn

from torch import optim

In [2]:
full_dataset = pd.read_csv('train.csv')
dataset = full_dataset[full_dataset['sc_w'] != 0].copy()

# Creating NN Model

Linear ->
ReLU ->
Out ->
Softmax

In [3]:
# class PricePredictor(nn.Module):
#     def __init__(self,input_size,hidden_size,out_size):
#         super(PricePredictor,self).__init__()
#         self.hidden = nn.Linear(input_size,hidden_size)
#         self.relu = nn.ReLU()
#         self.out = nn.Linear(hidden_size,out_size)
#         self.softmax = nn.Softmax()

#     def forward(self,X):
#         feature = self.relu(self.hidden(X))
#         output = self.softmax(self.out(feature))
#         return output

In [4]:
dataset_X = dataset.drop('price_range',axis=1).copy()
dataset_Y = dataset['price_range'].copy()

In [5]:
input_size = dataset_X.shape[1]
hidden_size = 32
out_size = len(set(dataset_Y))

In [6]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [7]:
# net = PricePredictor(input_size,hidden_size,out_size).to(device)

In [8]:
net = nn.Sequential(nn.Linear(input_size,hidden_size), nn.ReLU(), nn.Linear(hidden_size,out_size), nn.Softmax())
net = net.to(device)

In [9]:
# X_tns = torch.from_numpy(dataset_X.values).float().to(device)
# Y_tns = torch.from_numpy(dataset_Y.values).to(device)
X_tns = torch.FloatTensor(dataset_X.values).to(device)
Y_tns = torch.LongTensor(dataset_Y.values).to(device)


In [10]:
pred = net(X_tns)

<ipython-input-3-08caa9eb778b>:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(self.out(feature))


In [11]:
criterion = nn.CrossEntropyLoss().to(device)

In [12]:
optimizer = optim.Adam(net.parameters(),lr=1e-3,weight_decay = 1e-3)

Forward

In [13]:
pred = net(X_tns)
loss = criterion(pred,Y_tns)

<ipython-input-3-08caa9eb778b>:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(self.out(feature))


Backward

In [14]:
loss.backward()
optimizer.step()

In a Loop

In [15]:
for i in range(500):
    pred = net(X_tns)
    loss = criterion(pred,Y_tns)
    print(loss)
    loss.backward()
    optimizer.step()

<ipython-input-3-08caa9eb778b>:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(self.out(feature))


tensor(1.5620, grad_fn=<NllLossBackward0>)
tensor(1.5423, grad_fn=<NllLossBackward0>)
tensor(1.5183, grad_fn=<NllLossBackward0>)
tensor(1.4978, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.4959, grad_fn=<NllLossBackward0>)
tensor(1.49